In [128]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

In [129]:
df = pd.read_csv("D:\\visualization\\formula1\\data\\processed\\df_train_v3.csv")

In [130]:
df.year.unique()

array([2009, 2008, 2007, 2006, 2005, 2004, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int64)

In [131]:
df.head()

,raceId,driverId,constructorId,year,race_date,final_position,driver_points,completed_laps,fastest_lap,fastest_lap_speed,race_time_ms,qualifying_position,driver_standing_points,driver_wins,constructor_points,constructor_wins
0,1,1,1,2009,2009-03-29,20,0.0,58,39.0,214.455,NaN,15.0,NaN,NaN,NaN,NaN
1,1,2,2,2009,2009-03-29,10,0.0,58,48.0,216.245,5662869.0,11.0,0.0,0.0,0.0,0.0
2,1,3,3,2009,2009-03-29,6,3.0,58,48.0,217.668,5661506.0,5.0,3.0,0.0,3.0,0.0
3,1,4,4,2009,2009-03-29,5,4.0,58,53.0,215.199,5660663.0,12.0,4.0,0.0,4.0,0.0
4,1,5,1,2009,2009-03-29,19,0.0,0,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8550 entries, 0 to 8549
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   raceId                  8550 non-null   int64  
 1   driverId                8550 non-null   int64  
 2   constructorId           8550 non-null   int64  
 3   year                    8550 non-null   int64  
 4   race_date               8550 non-null   object 
 5   final_position          8550 non-null   int64  
 6   driver_points           8550 non-null   float64
 7   completed_laps          8550 non-null   int64  
 8   fastest_lap             8159 non-null   float64
 9   fastest_lap_speed       8159 non-null   float64
 10  race_time_ms            4235 non-null   float64
 11  qualifying_position     8526 non-null   float64
 12  driver_standing_points  8488 non-null   float64
 13  driver_wins             8488 non-null   float64
 14  constructor_points      8530 non-null   

## Clasificación

In [133]:
import numpy as np

# Asegúrate de que 'ds' sea la columna de fecha y 'prob_ganar' sea la variable objetivo (1 si ganó, 0 si no)
df['ds'] = pd.to_datetime(df['race_date']).dt.to_period('M').dt.to_timestamp()
df['ds'] = pd.to_datetime(df['ds'])
df['prob_ganar'] = np.where(df['final_position'] == 1, 1, 0)


# Imputar valores faltantes con la mediana por columna
df = df.fillna(df.median(numeric_only=True))

In [134]:
# Agrupar por piloto y fecha (esto ayuda a obtener la media de probabilidad si hay duplicados)
df = df.groupby(['driverId', 'ds']).agg({
    'prob_ganar': 'mean',
    'fastest_lap': 'mean',
    'driver_points': 'mean',
    'completed_laps': 'mean',
    'fastest_lap_speed': 'mean',
    'qualifying_position': 'mean',
    'driver_standing_points': 'mean',
    'constructor_points': 'mean',
    'constructor_wins': 'mean'
}).reset_index()

In [135]:
df.isnull().sum()

driverId                  0
ds                        0
prob_ganar                0
fastest_lap               0
driver_points             0
completed_laps            0
fastest_lap_speed         0
qualifying_position       0
driver_standing_points    0
constructor_points        0
constructor_wins          0
dtype: int64

In [136]:
# Diccionario para almacenar modelos y predicciones por piloto
modelos = {}
resultados_predicciones = []

# Iterar sobre cada piloto
for piloto in df['driverId'].unique():
    # Filtrar datos para el piloto actual
    df_piloto = df[df['driverId'] == piloto][['ds', 'prob_ganar', 
                                               'fastest_lap', 'driver_points', 'completed_laps',
                                               'fastest_lap_speed', 'qualifying_position',
                                               'driver_standing_points', 'constructor_points', 'constructor_wins']]
    
    # Renombrar 'prob_ganar' a 'y' para Prophet
    df_piloto = df_piloto.rename(columns={'prob_ganar': 'y'})
    
    # Verificar si hay al menos 2 filas no nulas en la columna 'y'
    if df_piloto['y'].notnull().sum() < 2:
        print(f"Piloto {piloto} tiene menos de 2 registros válidos. Saltando este piloto.")
        continue  # Saltar al siguiente piloto
    
    # Crear y configurar el modelo Prophet con regresores
    modelo = Prophet()
    modelo.add_regressor('fastest_lap')
    modelo.add_regressor('driver_points')
    modelo.add_regressor('completed_laps')
    modelo.add_regressor('fastest_lap_speed')
    modelo.add_regressor('qualifying_position')
    modelo.add_regressor('driver_standing_points')
    modelo.add_regressor('constructor_points')
    modelo.add_regressor('constructor_wins')
    
    # Entrenar el modelo
    modelo.fit(df_piloto)
    modelos[piloto] = modelo
    
    # Crear un DataFrame de fechas futuras hasta diciembre de 2024
    future = modelo.make_future_dataframe(periods=12, freq='M')
    future = future[future['ds'] <= '2024-12-31']
    
    # Añadir los valores de los regresores al DataFrame de fechas futuras
    future = future.merge(df_piloto[['ds', 'fastest_lap', 'driver_points', 'completed_laps',
                                     'fastest_lap_speed', 'qualifying_position',
                                     'driver_standing_points', 'constructor_points', 'constructor_wins']],
                          on='ds', how='left')
    
    # Rellenar nuevamente los valores nulos después del merge
    future = future.fillna(df_piloto.median(numeric_only=True))
    
    # Verificación adicional para confirmar que no haya valores nulos en future
    if future.isnull().values.any():
        print(f"Valores NaN encontrados en 'future' para piloto {piloto}. Saltando este piloto.")
        continue  # Saltar al siguiente piloto
    
    # Generar predicciones para el futuro
    forecast = modelo.predict(future)
    
    # Filtrar solo las predicciones para diciembre de 2024 usando mes y año
    forecast_diciembre_2024 = forecast[(forecast['ds'].dt.year == 2024) & (forecast['ds'].dt.month == 12)]
    
    # Verificar si hay predicciones para diciembre de 2024
    if forecast_diciembre_2024.empty:
        print(f"No hay predicciones para diciembre de 2024 para el piloto {piloto}.")
        continue
    
    forecast_diciembre_2024['driverId'] = piloto  # Agregar el ID del piloto
    
    # Guardar la predicción en la lista de resultados
    resultados_predicciones.append(forecast_diciembre_2024[['ds', 'yhat', 'driverId']])

# Combinar todas las predicciones en un solo DataFrame y ordenar por probabilidad de ganar
predicciones_totales = pd.concat(resultados_predicciones, ignore_index=True)
predicciones_totales = predicciones_totales.sort_values(by='yhat', ascending=False).reset_index(drop=True)


20:40:16 - cmdstanpy - INFO - Chain [1] start processing
20:40:16 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\cris_\AppData\Local\Temp\ipykernel_20800\3974405335.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_diciembre_2024['driverId'] = piloto  # Agregar el ID del piloto
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
20:40:16 - cmdstanpy - I

No hay predicciones para diciembre de 2024 para el piloto 2.


20:40:16 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
20:40:17 - cmdstanpy - INFO - Chain [1] start processing
20:40:17 - cmdstanpy - INFO - Chain [1] done processing


No hay predicciones para diciembre de 2024 para el piloto 3.


c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\cris_\AppData\Local\Temp\ipykernel_20800\3974405335.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_diciembre_2024['driverId'] = piloto  # Agregar el ID del piloto
20:40:17 - cmdstanpy - INFO - Chain [1] start processing
20:40:17 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
c:\Users\cris_\anaconda3

...

c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


No hay predicciones para diciembre de 2024 para el piloto 853.
No hay predicciones para diciembre de 2024 para el piloto 854.


c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\cris_\AppData\Local\Temp\ipykernel_20800\3974405335.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_diciembre_2024['driverId'] = piloto  # Agregar el ID del piloto
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
20:40:35 - cmdstanpy - INFO - Chain [1] start processing
20:40:35 - cmdstanpy - INFO - Chain [1] done processing


No hay predicciones para diciembre de 2024 para el piloto 856.


c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\cris_\AppData\Local\Temp\ipykernel_20800\3974405335.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_diciembre_2024['driverId'] = piloto  # Agregar el ID del piloto
c:\Users\cris_\anaconda3\envs\modelo_formula1\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\cris_\AppData\Local\Temp\ipykernel_20800\3974405335.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [137]:
# Aplicar transformación sigmoide para convertir yhat en probabilidad
predicciones_totales['probabilidad_ganar'] = 1 / (1 + np.exp(-predicciones_totales['yhat']))
predicciones_totales['probabilidad_ganar'] = predicciones_totales['probabilidad_ganar'].round(3)
# Ordenar por probabilidad de ganar y mostrar el resultado
predicciones_totales = predicciones_totales.sort_values(by='probabilidad_ganar', ascending=False).reset_index(drop=True)

In [138]:
predicciones_totales

,ds,yhat,driverId,probabilidad_ganar
0,2024-12-31,5.140886,846,0.994
1,2024-12-31,4.426892,844,0.988
2,2024-12-31,2.271836,839,0.907
3,2024-12-31,0.843665,847,0.699
4,2024-12-31,0.070796,1,0.518
5,2024-12-31,0.000000,848,0.500
6,2024-12-31,-0.001661,817,0.500
7,2024-12-31,0.000000,807,0.500
8,2024-12-31,0.000000,825,0.500
9,2024-12-31,0.000000,840,0.500


### Asignación de nombres

In [144]:
import mysql.connector

In [145]:
# Conexión a la base de datos MySQL
conn = mysql.connector.connect(
    host='localhost',       # Dirección del servidor MySQL (puede ser localhost si es local)
    user='root',            # Usuario de MySQL (ajusta según tu configuración)
    password='Ringochunin1302',  # Contraseña de MySQL
    database='ergast_f1'    # Nombre de la base de datos
)


# Crear un cursor para ejecutar la consulta
cursor = conn.cursor()

In [146]:
# Ejecutar la consulta para obtener los años disponibles
query = """
SELECT 
    d.driverId,
    d.forename AS driver_first_name,
    d.surname AS driver_last_name,
    c.constructorId,
    c.name AS constructor_name
FROM 
    drivers d
JOIN 
    results r ON d.driverId = r.driverId
JOIN 
    constructors c ON r.constructorId = c.constructorId
JOIN 
    races ra ON r.raceId = ra.raceId
WHERE 
    ra.year = (SELECT MAX(year) FROM races)  -- Filtrar para el año más reciente
GROUP BY 
    d.driverId, d.forename, d.surname, c.constructorId, c.name
ORDER BY 
    d.driverId;

"""


In [147]:
%%time
cursor.execute(query)

# Obtener los resultados y convertirlos en un DataFrame de pandas
result = cursor.fetchall()

# Obtener los nombres de las columnas
columns = [i[0] for i in cursor.description]

# Convertir los resultados en un DataFrame
df_names = pd.DataFrame(result, columns=columns)

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

CPU times: total: 0 ns
Wall time: 584 ms


In [154]:
df_names['name'] = df_names.driver_first_name + ' ' + df_names.driver_last_name + ' - ' + df_names.constructor_name

In [156]:
df_names

,driverId,driver_first_name,driver_last_name,constructorId,constructor_name,name
0,1,Lewis,Hamilton,131,Mercedes,Lewis Hamilton - Mercedes
1,4,Fernando,Alonso,117,Aston Martin,Fernando Alonso - Aston Martin
2,807,Nico,Hülkenberg,210,Haas F1 Team,Nico Hülkenberg - Haas F1 Team
3,815,Sergio,Pérez,9,Red Bull,Sergio Pérez - Red Bull
4,817,Daniel,Ricciardo,215,RB F1 Team,Daniel Ricciardo - RB F1 Team
5,822,Valtteri,Bottas,15,Sauber,Valtteri Bottas - Sauber
6,825,Kevin,Magnussen,210,Haas F1 Team,Kevin Magnussen - Haas F1 Team
7,830,Max,Verstappen,9,Red Bull,Max Verstappen - Red Bull
8,832,Carlos,Sainz,6,Ferrari,Carlos Sainz - Ferrari
9,839,Esteban,Ocon,214,Alpine F1 Team,Esteban Ocon - Alpine F1 Team


In [162]:
df_prediction = pd.merge(predicciones_totales,df_names[['driverId','name']],how = 'left', on='driverId')[['name','probabilidad_ganar']]

In [163]:
# Usar doble barra invertida en la ruta
df_prediction.to_csv("D:\\visualization\\formula1\\data\\processed\\df_prediction.csv", index=False)